# download spicy products for ASO rasters

This notebook contains functions to run spicy-snow for the region/date of each snow depth raster in a directory.

NOTES:
- Changed sentinel1.py in spicy to resample to 50 m pixel spacing (matching ASO) rather than default 90.
- Changed user_dates.py to have start date on September 1 instead of August 1

In [1]:
# # install spicy-snow in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install spicy-snow

In [2]:
# import spicy-snow functions and other required packages
# depending on your environment, you may need to install some of these
from spicy_snow.retrieval import retrieve_snow_depth
from spicy_snow.IO.user_dates import get_input_dates
import geopandas as gpd
from pathlib import Path
from shapely import geometry
from glob import glob
import rasterio as rio
import os

In [3]:
def spicy_for_aso(dir_path):
    '''
    Run spicy for the extent and date of ASO rasters
    '''
    path_list = glob(f'{dir_path}/*ASO_50M_SD*.tif')
    
    for i, path in enumerate(path_list):
        print(f'-------\nworking on {path.split("/")[-1]}, {i+1}/{len(path_list)}\n-------')
        # get bounding box from ASO raster
        raster = rio.open(path)
        gdf = gpd.GeoDataFrame({"id":1,"geometry":[geometry.box(*raster.bounds)]}, crs=raster.crs)
        area = gdf.to_crs('4326').geometry.values[0]
        
        # get acquistion date from ASO raster
        dates = get_input_dates(path.split('/')[-1].split('_')[-2])
        
        # define output directory and file name
        site_name = path.split('/')[-1].split('_')[-3]
        utm_zone = path.split('/')[-2]
        out_nc = Path(f'..data/spicy/{utm_zone}/spicy_{site_name}_{dates[0]}_{dates[1]}.nc').expanduser()
        
        if os.path.exists(out_nc):
            print('spicy-snow output already exists, skipping')
            
        else:
            # run spicy-snow for ASO raster
            spicy_ds = retrieve_snow_depth(area = area,
                                           dates = dates, 
                                           work_dir = Path('/tmp/spicy/').expanduser(), 
                                           job_name = f'spicy_{site_name}_{dates[0]}_{dates[1]}',
                                           existing_job_name = f'spicy_{site_name}_{dates[0]}_{dates[1]}',
                                           debug=False,
                                           outfp=out_nc)
            
            #!rm /tmp/spicy/tmp/S1*.tif

In [ ]:
path = '/mnt/c/Users/qbren/Desktop/taco/projects/crunchy-snow/data/ASO/ASO_50m_SD_cleaned/utm12n'
spicy_for_aso(path)

-------
working on ASO_50M_SD_Dolores_20210420_clean.tif, 1/8
-------
(spicy-snow spicy_snow.utils.spicy_logging INFO) Found 115 results


Submitting s1 jobs: 100%|███████████████████████████████████████████████████| 115/115 [01:09<00:00,  1.65it/s]

(spicy-snow spicy_snow.utils.spicy_logging INFO) Watching 115 jobs. This may take a while...


  0%|          | 0/115 [timeout in 10800 s]

Combining Sentinel-1 dataArrays: 100%|██████████████████████████████████████| 115/115 [07:20<00:00,  3.83s/it]